In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import statsmodels.api as sm
#import sklearn as sk
import re #import regex
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import mean_squared_error
import scipy.stats
import datetime

**Note**: There isn't a separate cleaning notebook for the ZCruit data because the only cleaning operation needed is to fix height values. That is done at the beginning of the `Handling Empty Values [FINAL]` notebook.

### Read in athletic.net Track & Field Data
We have called the original track and field data, `original_tf_data.csv` and we load it in as the variable, `tf_data`.

In [3]:
tf_data = pd.read_csv("original_tf_data.csv", sep=",")
tf_data.head(5)

,Zcruit ID,Player Name,Grad Year,High School Name,High School City,High School State,4 x 100 Meter,4 x 100 Meter Event,4 x 100 Meter Occurred on,4 x 200 Meter,...,Javelin Occurred on,Discus,Discus Event,Discus Occurred on,Pole Vault,Pole Vault Event,Pole Vault Occurred on,Hammer,Hammer Event,Hammer Occurred on
0,7b8c6df6-b548-4ed4-ae9b-4a741d7746c6,Bryson Lightfoot,2021,Birdville High School,Haltom City,TX,43.29s,UIL 5A District 07 (W-Th) Track Meet,4/3/2019,98.08s,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,45be7538-29ad-4e4a-9d2e-be05b0d3f595,Jalik Lewis,2022,Denison Senior High School,Denison,TX,44.42s,Liberty Christian Bubba Joiner Invitational (F...,3/5/2020,96.86s,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ae1749c5-a0f1-4fbc-852b-1ac51f5b1404,Jaiden Brown,2022,Southern Regional 9-10 High School,Manahawkin,NJ,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,eb32fcc6-a8f5-41df-b524-a4fc824a14f3,Mark Phifer,2021,Santa Monica High School,Santa Monica,CA,43.58s,Mira Costa vs Santa Monica Dual Meet,2/27/2020,96.31s,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9712f1c5-6319-49ba-b8df-70b4102d3737,Gerimiah Brown,2021,Vaca High School,Vacaville,CA,45.02s,Stocking Super Seven Invitational,2/29/2020,97.20s,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Define 2 main helper functions. 
1) `strip`: Cleans track events (100M, 200M, hurdles). Removes "s" for seconds, (1.8) for wind speeds, "cs" for alternative measurements

2) `str_to_val`: Cleans field events (high jump, long jump, triple jump, shotput, discus). Converts 5' 6" to 66.0. Also removes "FOUL", "m" for meters, "-" as a dash. 

Both functions also replace outlier values as empty values. For example, assume a high jump over 10 feet is a data entry error and replace it with an empty value to be filled in with the master filling in empty values function in `Handling Empty Values [FINAL]` notebook.

In [4]:
#Function to convert data into float types for track events
def strip(time):
    """Takes in a time for a track event (100M, 200M, 400M, 110M Hurdles, 300M Hurdles)
    
        Returns a cleaned version of the time as a float.
        
        Input: 12.5 (1.8)s
        Outputs: 12.5
        """
    if type(time) == float:
        return time
    cs_away = time.rstrip(' cs')
    space_away = cs_away.split(" ")[0]
    paran_away = cs_away.split("(")[0]
    if len(space_away) == 5:
        return float(space_away)
    elif len(paran_away) == 5:
        return float(paran_away)
    
#Function to convert data into float types for field events
height_regex = "^(\d)'\s*(\d*\.*\d*)"
def str_to_val(height_str, event):
    """Takes in a mark for a field event (High Jump, Long Jump, Triple Jump, Shotput, Discus) - typically measured as ft' inches".
        
        Returns a cleaned version of that mark as a float in inches. 
        
        Handle each event separately for event-specific inconsistencies. 
        There is a lot of repeated code, so a lot of this can be generalized
        
        Input: 35' 10.75"
        Output: 430.75
    """
    if type(height_str) == float:
            return height_str
    if event == 'High Jump':
        # We want to case this one for the possibility of reported value being in meters instead of ft, inches
        if not("'" in height_str):
            m_float =  float(height_str.split('"')[0])
            # The conversion for meter to inch is 39.3701 per Google Search
            inch_float = m_float*39.3701
            return inch_float
        else:
            match = re.match("^(\d*)'\s*(\d*\.*\d*)", height_str)
            groups = match.groups()
            feet, inches = int(groups[0]), float(groups[1])
            if (feet > 10) | (feet < 3):
                return np.nan
            return feet * 12 + inches
    elif event == 'Long Jump':
        # We want to case this one for the possibility of reported value being in meters instead of ft, inches
        if "m" in height_str:
            m_float =  float(height_str.split('m')[0])
            inch_float = m_float*39.3701
            return inch_float
        elif "FOUL" in height_str:
            return np.nan
        elif not("'" in height_str):
            m_float =  float(height_str.split('"')[0])
            # The conversion for meter to inch is 39.3701 per Google Search
            inch_float = m_float*39.3701
            return inch_float
        else:
            match = re.match("^(\d*)'\s*(\d*\.*\d*)", height_str)
            groups = match.groups()
            feet, inches = int(groups[0]), float(groups[1])
            if feet < 10:
                return np.nan
            return feet * 12 + inches
    elif event == 'Triple Jump':
        # We want to case this one for the possibility of reported value being in meters instead of ft, inches
        if "m" in height_str:
            m_float =  float(height_str.split('m')[0])
            inch_float = m_float*39.3701
            return inch_float
        elif "FOUL" in height_str:
            return np.nan
        elif not("'" in height_str):
            m_float =  float(height_str.split('"')[0])
            # The conversion for meter to inch is 39.3701 per Google Search
            inch_float = m_float*39.3701
            return inch_float
        else:
            match = re.match("^(\d*)'\s*(\d*\.*\d*)", height_str)
            groups = match.groups()
            feet, inches = int(groups[0]), float(groups[1])
            # This 680 is the collegiate record of Triple Jump -> High Schoolers can't pass this
            if (feet*12 + inches > 680) | (feet < 15):
                return np.nan
            return feet * 12 + inches
    elif event == "Shotput":
        # We want to case this one for the possibility of reported value being in meters instead of ft, inches
        if "m" in height_str:
            m_float =  float(height_str.split('m')[0])
            inch_float = m_float*39.3701
            return inch_float
        elif "FOUL" in height_str:
            return np.nan
        elif "-" in height_str:
            return np.nan
        elif ('""' in height_str) | ('"""' in height_str) | ('""""' in height_str):
            # CAN PARSE (ROOM FOR IMPROVEMENT) -> i.e. "30' 9""""" can be parsed
            return np.nan
        elif not("'" in height_str):
            m_float =  float(height_str.split('"')[0])
            # The conversion for meter to inch is 39.3701 per Google Search
            inch_float = m_float*39.3701
            return inch_float
        else:
            match = re.match("^(\d*)'\s*(\d*\.*\d*)", height_str)
            groups = match.groups()
            feet, inches = int(groups[0]), float(groups[1])
            if feet < 10:
                return np.nan
            return feet * 12 + inches
    elif event == "Discus":
        # We want to case this one for the possibility of reported value being in meters instead of ft, inches
        if "m" in height_str:
            m_float =  float(height_str.split('m')[0])
            inch_float = m_float*39.3701
            return inch_float
        elif "FOUL" in height_str:
            return np.nan
        elif "-" in height_str:
            return np.nan
        elif ('""' in height_str) | ('"""' in height_str) | ('""""' in height_str):
            # CAN PARSE (ROOM FOR IMPROVEMENT) -> i.e. "30' 9""""" can be parsed
            return np.nan
        elif not("'" in height_str):
            m_float =  float(height_str.split('"')[0])
            # The conversion for meter to inch is 39.3701 per Google Search
            inch_float = m_float*39.3701
            if inch_float < 360:
                return np.nan
            return inch_float
        else:
            match = re.match("^(\d*)'\s*(\d*\.*\d*)", height_str)
            groups = match.groups()
            feet, inches = int(groups[0]), float(groups[1])
            if feet < 30:
                return np.nan
            return feet * 12 + inches

### Define master cleaning function called master_tf_cleaning. 
Takes in a raw track and field dataset such as `tf_data` and spits out a cleaned dataset of the 8 events we wanted (100M, 200M, 110M Hurdles, 300M Hurdles, High Jump, Long Jump, Triple Jump, Shotput, Discus). Feel free to edit what events you want to include.

In [5]:
def master_tf_cleaning(data):
    """Takes in the raw track & field data.
        Returns a cleaned track & field dataset with ZCruit ID, Player Name, and the 8 events described above the cell.
    """
    data = data[['Zcruit ID', 'Player Name', '100 Meter', '200 Meter', '110 Meter Hurdles', '300 Meter Hurdles', 'High Jump', 'Long Jump', 'Triple Jump', 'Shotput', 'Discus']]
    track_events = ['100 Meter', '200 Meter', '110 Meter Hurdles', '300 Meter Hurdles']
    field_events = ['High Jump', 'Long Jump', 'Triple Jump', 'Shotput', 'Discus']
    for i in track_events:
        data[i] = data.apply(lambda x: strip(x[i]), axis=1)
    for i in field_events:
        data[i] = data.apply(lambda x: str_to_val(x[i], i), axis=1)
    return data

Apply `master_tf_cleaning` to `tf_data` and store resulting dataset into `cleaned_tf_data`

In [6]:
cleaned_tf_data = master_tf_cleaning(tf_data)

<ipython-input-5-2129b824a5b5>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[i] = data.apply(lambda x: strip(x[i]), axis=1)
<ipython-input-5-2129b824a5b5>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[i] = data.apply(lambda x: str_to_val(x[i], i), axis=1)


Show first 5 rows of cleaned data. As you can see, the event marks are all floats of either seconds (for track events) or inches (for field events) and empty values have not been filled in. This cleaned dataset has been saved as `clean_tf_data.csv` which is used in future notebooks such as `Handling Empty Values [FINAL]` which handles the empty values.

In [7]:
cleaned_tf_data.head(5)

,Zcruit ID,Player Name,100 Meter,200 Meter,110 Meter Hurdles,300 Meter Hurdles,High Jump,Long Jump,Triple Jump,Shotput,Discus
0,7b8c6df6-b548-4ed4-ae9b-4a741d7746c6,Bryson Lightfoot,11.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,45be7538-29ad-4e4a-9d2e-be05b0d3f595,Jalik Lewis,NaN,24.95,NaN,NaN,66.0,217.0,NaN,NaN,NaN
2,ae1749c5-a0f1-4fbc-852b-1ac51f5b1404,Jaiden Brown,NaN,25.90,NaN,NaN,NaN,NaN,NaN,490.5,NaN
3,eb32fcc6-a8f5-41df-b524-a4fc824a14f3,Mark Phifer,12.51,24.94,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9712f1c5-6319-49ba-b8df-70b4102d3737,Gerimiah Brown,11.74,25.77,NaN,47.84,NaN,207.5,NaN,NaN,NaN


### Load in the already cleaned track and field dataset.
We save the following cleaned track and field dataset as `cleaned_tf_data.csv`. To get the cleaned dataset without doing the above work, you can read in the `cleaned_tf_data.csv` as shown below. Show the first 5 rows. This dataset should look exactly the same as the one produced from the above work.

In [8]:
pre_loaded_cleaned_tf_data = pd.read_csv('cleaned_tf_data.csv')
pre_loaded_cleaned_tf_data = pre_loaded_cleaned_tf_data.drop(columns = 'Unnamed: 0')
pre_loaded_cleaned_tf_data.head(5)

,Zcruit ID,Player Name,100 Meter,200 Meter,110 Meter Hurdles,300 Meter Hurdles,High Jump,Long Jump,Triple Jump,Shotput,Discus
0,7b8c6df6-b548-4ed4-ae9b-4a741d7746c6,Bryson Lightfoot,11.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,45be7538-29ad-4e4a-9d2e-be05b0d3f595,Jalik Lewis,NaN,24.95,NaN,NaN,66.0,217.0,NaN,NaN,NaN
2,ae1749c5-a0f1-4fbc-852b-1ac51f5b1404,Jaiden Brown,NaN,25.90,NaN,NaN,NaN,NaN,NaN,490.5,NaN
3,eb32fcc6-a8f5-41df-b524-a4fc824a14f3,Mark Phifer,12.51,24.94,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9712f1c5-6319-49ba-b8df-70b4102d3737,Gerimiah Brown,11.74,25.77,NaN,47.84,NaN,207.5,NaN,NaN,NaN
